# Determining the CDS Basket Spread

An analysis of pricing a CDS Basket 

## NOTE THAT THE API WILL CHANGE BUT THE UNDERLYING MODELS WILL REMAIN

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [1]:
from financepy.products.credit import *
from financepy.products.libor import *
from financepy.finutils import *

###################################################################
# FINANCEPY BETA Version 0.180 - This build: 22 Sep 2020 at 22:13 #
#     This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy #
###################################################################



In [2]:
tradeDate = FinDate(1, 8, 2007)
settlementDate = tradeDate.addWorkDays(1)

## Build Libor Curve

In [6]:
depos = []
dcType = FinDayCountTypes.THIRTY_E_360_ISDA
fixedFreq = FinFrequencyTypes.SEMI_ANNUAL
swapType = FinLiborSwapTypes.PAYER
swap1 = FinLiborSwap(settlementDate,"1Y",swapType,0.0502,fixedFreq,dcType)
swap2 = FinLiborSwap(settlementDate,"2Y",swapType,0.0502,fixedFreq,dcType)
swap3 = FinLiborSwap(settlementDate,"3Y",swapType,0.0501,fixedFreq,dcType)
swap4 = FinLiborSwap(settlementDate,"4Y",swapType,0.0502,fixedFreq,dcType)
swap5 = FinLiborSwap(settlementDate,"5Y",swapType,0.0501,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5]

liborCurve = FinLiborCurve(settlementDate, depos, [], swaps)

We treat an index as a CDS contract with a flat CDS curve at the CDS index spread for the same maturity

## Create the Underlying CDS Index Portfolio

In [8]:
stepInDate = tradeDate.addWorkDays(1)
valuationDate = stepInDate

In [9]:
maturity3Y = tradeDate.nextCDSDate(36)
maturity5Y = tradeDate.nextCDSDate(60)
maturity7Y = tradeDate.nextCDSDate(84)
maturity10Y = tradeDate.nextCDSDate(120)

### Heterogeneous Curves

In [10]:
f = open('.//data//CDX_NA_IG_S7_SPREADS.csv', 'r')
data = f.readlines()
heteroIssuerCurves = []

numCredits = len(data) - 1  # The file has a header

for row in data[1:]:
    splitRow = row.split(",")
    spd3Y = float(splitRow[1]) / 10000.0
    spd5Y = float(splitRow[2]) / 10000.0
    spd7Y = float(splitRow[3]) / 10000.0
    spd10Y = float(splitRow[4]) / 10000.0
    recoveryRate = float(splitRow[5])
    cds3Y = FinCDS(stepInDate, maturity3Y, spd3Y)
    cds5Y = FinCDS(stepInDate, maturity5Y, spd5Y)
    cds7Y = FinCDS(stepInDate, maturity7Y, spd7Y)
    cds10Y = FinCDS(stepInDate, maturity10Y, spd10Y)
    cdsContracts = [cds3Y, cds5Y, cds7Y, cds10Y]
    issuerCurve = FinCDSCurve(valuationDate, cdsContracts, liborCurve, recoveryRate)
    heteroIssuerCurves.append(issuerCurve)

### Homogeneous Curves 

Calculate the average spread of the heterogeneous portfolio

In [11]:
homoIssuerCurves = []
numCredits = 125
recoveryRate = 0.40 

In [12]:
cdsIndex = FinCDSIndexPortfolio()

In [13]:
spd3Y = cdsIndex.intrinsicSpread(valuationDate, stepInDate, maturity3Y, heteroIssuerCurves)
spd5Y = cdsIndex.intrinsicSpread(valuationDate, stepInDate, maturity5Y, heteroIssuerCurves)
spd7Y = cdsIndex.intrinsicSpread(valuationDate, stepInDate, maturity7Y, heteroIssuerCurves)
spd10Y = cdsIndex.intrinsicSpread(valuationDate, stepInDate, maturity10Y, heteroIssuerCurves)

In [14]:
print("Homogeneous curve 3Y:", spd3Y*10000)
print("Homogeneous curve 5Y:", spd5Y*10000)
print("Homogeneous curve 7Y:", spd7Y*10000)
print("Homogeneous curve 10Y:", spd10Y*10000)

Homogeneous curve 3Y: 19.678938015086928
Homogeneous curve 5Y: 35.539296718015336
Homogeneous curve 7Y: 49.012038203822
Homogeneous curve 10Y: 61.41424224623194


In [15]:
for row in range(0,numCredits):
    cds3Y = FinCDS(stepInDate, maturity3Y, spd3Y)
    cds5Y = FinCDS(stepInDate, maturity5Y, spd5Y)
    cds7Y = FinCDS(stepInDate, maturity7Y, spd7Y)
    cds10Y = FinCDS(stepInDate, maturity10Y, spd10Y)
    cdsContracts = [cds3Y, cds5Y, cds7Y, cds10Y]
    issuerCurve = FinCDSCurve(valuationDate, cdsContracts, liborCurve, recoveryRate)
    homoIssuerCurves.append(issuerCurve)

## Define the Baskets

In [16]:
numCredits = 5

In [17]:
issuerCurves = heteroIssuerCurves[0:numCredits]

In [18]:
basketMaturity = FinDate(2011, 12, 20)
cdsIndex = FinCDSIndexPortfolio()

In [19]:
intrinsicSpd = cdsIndex.intrinsicSpread(valuationDate, stepInDate, basketMaturity, issuerCurves) * 10000.0

print("INTRINSIC SPD BASKET MATURITY", intrinsicSpd)

totalSpd = cdsIndex.totalSpread(valuationDate, stepInDate, basketMaturity, issuerCurves) * 10000.0

print("SUMMED UP SPD BASKET MATURITY", totalSpd)

minSpd = cdsIndex.minSpread(valuationDate, stepInDate, basketMaturity, issuerCurves) * 10000.0

print("MINIMUM SPD BASKET MATURITY", minSpd)

maxSpd = cdsIndex.maxSpread(valuationDate, stepInDate, basketMaturity, issuerCurves) * 10000.0

print("MAXIMUM SPD BASKET MATURITY", maxSpd)

INTRINSIC SPD BASKET MATURITY 29.03670162944296
SUMMED UP SPD BASKET MATURITY 145.80336927009776
MINIMUM SPD BASKET MATURITY 9.61998164104911
MAXIMUM SPD BASKET MATURITY 73.20625604728323


In [20]:
basket = FinCDSBasket(valuationDate,basketMaturity)

## Gaussian Copula Model

WARNING: THE INTERFACE ON THIS FUNCTION WILL CHANGE SOON! THEY ALSO NEED TO BE ADAPTED TO USE NUMBA.

In [21]:
seed = 42
doF = 5
numTrials = 5000

print("NTrials   Rho    NTD    SPD_GC_MC    SPD_1FGC    SPD_ST10")
print("=========================================================")

for ntd in range(1, numCredits + 1):
    for beta in [0.0, 0.25, 0.5, 0.75, 0.90, 0.9999]:
        rho = beta * beta
        betaVector = np.ones(numCredits) * beta
        corrMatrix = corrMatrixGenerator(rho, numCredits)
        for numTrials in [5000]:
            v1 = basket.valueGaussian_MC(valuationDate,ntd,issuerCurves,corrMatrix,liborCurve,numTrials,seed)
            v2 = basket.value1FGaussian_Homo(valuationDate,ntd,issuerCurves,betaVector,liborCurve)
            v3 = basket.valueStudentT_MC(valuationDate, ntd, issuerCurves, corrMatrix, doF, liborCurve, numTrials,seed)
            print("%7d  %5.2f    %d    %9.3f   %9.3f    %9.3f"% (numTrials, rho, ntd, v1[2] * 10000, v2[3] * 10000, v3[2] *10000))
    print("=========================================================")


NTrials   Rho    NTD    SPD_GC_MC    SPD_1FGC    SPD_ST10
   5000   0.00    1      136.548     144.123      124.471
   5000   0.06    1      131.953     142.104      121.588
   5000   0.25    1      122.228     133.641      115.072
   5000   0.56    1      105.716     111.975       96.796
   5000   0.81    1       83.155      88.634       81.201
   5000   1.00    1       70.956      73.207       68.552
   5000   0.00    2        4.193       5.054       14.679
   5000   0.06    2        6.282       6.820       16.535
   5000   0.25    2       13.686      13.436       21.468
   5000   0.56    2       24.856      25.496       27.767
   5000   0.81    2       30.658      32.098       29.977
   5000   1.00    2       20.593      22.006       20.717
   5000   0.00    3        0.122       0.086        1.558
   5000   0.06    3        0.377       0.219        2.193
   5000   0.25    3        1.266       1.451        4.468
   5000   0.56    3        7.599       7.505       10.607
   5000   0.81

Copyright (c) 2020 Dominic O'Kane